In [2]:
import os
os.getcwd()



'/home/dang/experiment/Face_Attribute/light_model'

In [3]:
from face_model import MobileFaceNet, l2_norm
from torchvision import transforms
from torchsummary import summary
import torch 
from torchvision import transforms
from PIL import Image

device = 'cuda'
img_path = '/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
img = Image.open(img_path).convert('RGB')  # 이미지 열기 및 RGB 포맷으로 변환

# Torch 텐서로 변환하기 위해 Torch 변환기 사용
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # 이미지 크기를 모델의 입력 크기에 맞게 조정
    transforms.ToTensor(),           # 이미지를 Torch 텐서로 변환
])

# 이미지를 Torch 텐서로 변환하고 배치 크기를 16으로 설정
batch_img_tensor = torch.stack([transform(img) for _ in range(16)]).to(device)
# batch_img_tensor = batch_img_tensor.permute(0,2,3,1)
print("input",batch_img_tensor.size())


model = MobileFaceNet(512).to(device)  # embeding size is 512 (feature vector)
model.load_state_dict(torch.load('Weights/MobileFace_Net', map_location=lambda storage, loc: storage))

model.fc = torch.nn.Linear(in_features= 512, out_features= 2048 * 4 * 4, bias=True).to(device)



# for param in model.parameters():
#     param.requires_grad = True
        
with torch.no_grad():
    embedding = model(batch_img_tensor)
    embedding = model.fc(embedding)
    print("embedding",embedding.size())
        
    embedding = embedding.view(-1, 2048, 4, 4)
        
print("embedding",embedding.size())

summary(model, (3,112,112))

input torch.Size([16, 3, 112, 112])
embedding torch.Size([16, 32768])
embedding torch.Size([16, 2048, 4, 4])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]           1,728
       BatchNorm2d-2           [-1, 64, 56, 56]             128
             PReLU-3           [-1, 64, 56, 56]              64
        Conv_block-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]             576
       BatchNorm2d-6           [-1, 64, 56, 56]             128
             PReLU-7           [-1, 64, 56, 56]              64
        Conv_block-8           [-1, 64, 56, 56]               0
            Conv2d-9          [-1, 128, 56, 56]           8,192
      BatchNorm2d-10          [-1, 128, 56, 56]             256
            PReLU-11          [-1, 128, 56, 56]             128
       Conv_block-12          [-1, 128, 56, 56]           

In [3]:
import pandas as pd
all_df = pd.read_csv("/home/dang/experiment/Face_Attribute/dataset/final_data_exep3.csv")
all_df

,id,eyeglass,blur,mask,young,no_beard,brown_hair,gray_hair,black_hair,state
0,celeba/img_celeba/157411.jpg,0,0,0,1,1,0,0,1,valid
1,celeba/img_celeba/000860.jpg,0,0,0,0,1,0,0,1,train
2,celeba/img_celeba/115232.jpg,0,0,0,1,1,1,0,0,valid
3,celeba/img_celeba/102586.jpg,0,0,0,1,1,0,0,0,valid
4,celeba/img_celeba/089477.jpg,1,0,0,0,0,0,1,0,test
...,...,...,...,...,...,...,...,...,...,...
10794,korean/Middle_Resolution/19101441/S005/L2/E02/...,0,0,0,1,1,0,0,0,valid
10795,korean/Middle_Resolution/19101442/S005/L2/E02/...,0,0,0,1,1,0,0,0,test
10796,korean/Middle_Resolution/19101443/S005/L2/E02/...,0,0,0,0,1,0,0,0,train
10797,korean/Middle_Resolution/19101512/S005/L2/E02/...,0,0,0,0,1,0,0,0,train


In [7]:
import tensorflow as tf
import torch
from torchvision import transforms
from PIL import Image
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.models import Model


img_path = '/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
img = Image.open(img_path).convert('RGB')  # 이미지 열기 및 RGB 포맷으로 변환

# Torch 텐서로 변환하기 위해 Torch 변환기 사용
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # 이미지 크기를 모델의 입력 크기에 맞게 조정
    transforms.ToTensor(),           # 이미지를 Torch 텐서로 변환
])

img = transform(img)

print(img)

<PIL.Image.Image image mode=RGB size=112x112 at 0x7FCA861D4F90>


In [35]:
import tensorflow as tf
import torch
from torchvision import transforms
from PIL import Image
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.models import Model
# new_output_layer = Dense(num_classes, activation='softmax')(pretrained_model.output)

# model_path = "/home/dang/experiment/Face_Attribute/light_model/GhostFaceNet_W1.3_S1_ArcFace.h5"

# # 모델 불러오기
# pretrained_model = tf.keras.models.load_model(model_path)
# for layer in pretrained_model.layers:
#     layer.trainable = False

# # 새로운 출력 레이어 추가
# new_output_layer = Dense(2048 * 4 * 4, activation=None)(pretrained_model.layers[-2].output)
# reshaped_output = Reshape((2048, 4, 4))(new_output_layer)
# # 새로운 모델 생성
# new_model = tf.keras.Model(inputs=pretrained_model.input, outputs=reshaped_output)

img_path = '/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
img = Image.open(img_path).convert('RGB')  # 이미지 열기 및 RGB 포맷으로 변환

# Torch 텐서로 변환하기 위해 Torch 변환기 사용
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # 이미지 크기를 모델의 입력 크기에 맞게 조정
    transforms.ToTensor(),           # 이미지를 Torch 텐서로 변환
])

# 이미지를 Torch 텐서로 변환하고 배치 크기를 16으로 설정
batch_img_tensor = torch.stack([transform(img) for _ in range(16)])
batch_img_tensor = batch_img_tensor.permute(0,2,3,1)
print("input",batch_img_tensor.size())

# 이미지를 모델에 전달하여 임베딩 추출
# with torch.no_grad():
# embedding = pretrained_model(batch_img_tensor.numpy())  # 배치 크기 16으로 설정

# 새로운 출력 레이어 추가 및 출력형태 변경
new_output_layer = Dense(2048 * 4 * 4, activation=None)(pretrained_model.output)
reshaped_output = Reshape((2048, 4, 4), name="reshape_expanded_output")(new_output_layer)

# 새로운 모델 생성
pretrained_model = Model(inputs=pretrained_model.input, outputs=reshaped_output)

# 모델 요약 정보 출력
embedding = pretrained_model(batch_img_tensor.numpy())  # 배치 크기 16으로 설정
embedding = torch.from_numpy(embedding.numpy())
# # 추출된 임베딩 값 출력
print(embedding)
# print(new_model.input)
# print(new_model.output)


input torch.Size([16, 112, 112, 3])
tensor([[[[-0.3151, -0.1369, -0.2893, -0.3378],
          [ 0.3407, -0.0095, -0.0523,  0.2280],
          [ 0.3465, -0.2254, -0.2300, -0.2777],
          [-0.1365, -0.4945,  0.2554, -0.2778]],

         [[ 0.2963, -0.4498,  0.0592,  0.0238],
          [ 0.2434,  0.0811, -0.1437,  0.1375],
          [ 0.0680,  0.2239,  0.3191, -0.2160],
          [ 0.1430,  0.0036, -0.3666, -0.1490]],

         [[-0.2251, -0.0595, -0.1659, -0.2752],
          [ 0.1307, -0.4907,  0.3292, -0.2124],
          [ 0.1635,  0.1276, -0.0061, -0.3074],
          [-0.1474, -0.0124,  0.0941,  0.0798]],

         ...,

         [[-0.2168,  0.0150,  0.2639,  0.0900],
          [-0.0104,  0.1052,  0.1239,  0.5073],
          [ 0.4493,  0.3690,  0.1567,  0.0871],
          [ 0.3225, -0.0212,  0.5386,  0.6345]],

         [[ 0.0585, -0.2195, -0.1511, -0.2193],
          [-0.3027, -0.3595,  0.2243, -0.5139],
          [-0.2143, -0.1592,  0.6806,  0.4672],
          [ 0.1356,  0.0037, 

In [21]:

# 이미지 전처리 함수 정의
preprocess = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 모델 경로 정의
model_path = "/home/dang/experiment/Face_Attribute/light_model/GhostFaceNet_W1.3_S1_ArcFace.h5"

# 모델 불러오기
pretrained_model = torch.load(model_path)
pretrained_model.eval()  # 모델을 추론 모드로 설정

# 이미지 불러오기 및 전처리
img_path = '/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
img = Image.open(img_path)
img_tensor = preprocess(img)
img_tensor = img_tensor.unsqueeze(0)  # 배치 차원 추가

# 이미지를 모델에 전달하여 임베딩 추출
with torch.no_grad():
    embedding = pretrained_model(img_tensor)

# 추출된 임베딩 값 출력
print(embedding.shape)

UnpicklingError: invalid load key, 'H'.

In [1]:
img_path = '/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
img = Image.open(img_path)
img

NameError: name 'Image' is not defined

In [ ]:
img_path = '/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
img = image.load_img(img_path, target_size=(112, 112))  # 이미지 크기를 모델의 입력 크기에 맞게 조정
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array)  # 이미지를 모델이 처리할 수 있는 형태로 전처리

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

model_path = "/home/dang/experiment/Face_Attribute/light_model/GhostFaceNet_W1.3_S1_ArcFace.h5"

# 모델 불러오기
pretrained_model = tf.keras.models.load_model(model_path)
for layer in pretrained_model.layers:
    layer.trainable = False

img_path = '/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
img = image.load_img(img_path, target_size=(112, 112))  # 이미지 크기를 모델의 입력 크기에 맞게 조정
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array)  # 이미지를 모델이 처리할 수 있는 형태로 전처리

# 이미지의 차원을 (1, 112, 112, 3)으로 조정
img_array = tf.expand_dims(img_array, axis=0)

# 이미지를 모델에 전달하여 임베딩 추출
embedding = pretrained_model.predict(img_array)  # 모델은 배치 크기를 포함하여 입력을 받기 때문에 차원을 확장해야 함

# 추출된 임베딩 값 출력
print(embedding.shape)

1/1 [==============================] - 1s 949ms/step
(1, 512)


In [11]:
embedding

array([[ 2173447.2  ,  2281672.8  ,  2618006.5  , -1057389.1  ,
        -2964086.5  ,  1266245.   ,  1660397.9  ,   -38533.26 ,
         -668437.2  ,  2217710.   ,  2418163.5  ,  -329161.8  ,
        -1340420.4  ,  2506024.   , -1911025.6  , -3275085.   ,
         -231236.23 ,  5201268.   ,  -199133.52 , -1369497.9  ,
          -39054.586,  1934425.   ,  3095482.2  ,  1226174.5  ,
        -2147393.   ,   909657.2  ,  1264038.8  , -1871674.6  ,
         -449456.2  , -2250681.5  ,  -539718.6  , -2624247.2  ,
        -1000622.9  ,  -811041.25 , -1399747.9  , -4684832.5  ,
         -390283.9  , -1070411.1  , -3543803.8  , -1061455.5  ,
         -601924.4  ,  -442572.34 ,   -75409.63 ,  -244802.97 ,
        -1007435.94 ,   893549.94 ,  1150805.4  , -4556031.   ,
        -2251285.   ,  2223308.5  ,   322983.3  ,  -418951.7  ,
          617478.06 ,  1230796.6  ,    50428.426,  2201989.8  ,
        -1901703.1  ,  -292460.9  ,  -949497.6  , -1066705.2  ,
         1535830.6  , -2467821.   ,  205

In [8]:
# 모델의 입력 레이어에 해당하는 Input 레이어 생성
import tensorflow as tf
input_shape = (112, 112, 3)
input_layer = tf.keras.layers.Input(shape=input_shape)
output = model(input_layer)
model = tf.keras.Model(inputs=input_layer, outputs=output)

# 모델의 입력 확인
print(model.input)

KerasTensor(type_spec=TensorSpec(shape=(None, 112, 112, 3), dtype=tf.float32, name='input_7'), name='input_7', description="created by layer 'input_7'")


<KerasTensor: shape=(None, 112, 112, 3) dtype=float32 (created by layer 'input_1')>

In [47]:

img_path='/home/dang/experiment/Face_Attribute/dataset/celeba/img_celeba/157411.jpg'
result = DeepFace.find(
  img_path = '157411.jpg',
  db_path = img_path,
  model_name = 'GhostFaceNet'
)

KeyboardInterrupt: 

In [37]:
from backbones.GhostFaceNets.backbones import ghostv2

model = ghostv2.GhostNetV2(stem_width=16,
                        stem_strides=1,
                        width_mul=1.3,
                        num_ghost_module_v1_stacks=2,  # num of `ghost_module` stcks on the head, others are `ghost_module_multiply`, set `-1` for all using `ghost_module`
                        input_shape=(112, 112, 3),
                        num_classes=0,
                        activation="prelu",
                        classifier_activation=None,
                        dropout=0,
                        pretrained=None,
                        model_name="ghostnetv2")

# model.summary()
model = keras.models.load_model("/home/dang/experiment/Face_Attribute/light_model/GhostFaceNet_W1.3_S1_ArcFace.h5")


In [27]:
import GhostFaceNets
# import torchsummary
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)
# basic_model = keras.models.load_model("/home/dang/experiment/Face_Attribute/light_model/GhostFaceNet_W1.3_S1_ArcFace.h5")

# torchsummary(basic_model, input_size=(3, 112, 112)) 

/root/anaconda3/envs/dang/lib/python3.11/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: activation_432 --> activation_432
>>>> Convert ReLU: activation_433 --> activation_433
>>>> Convert ReLU: activation_434 --> activation_434
>>>> Convert ReLU: activation_435 --> activation_435
>>>> Convert ReLU: activation_436 --> activation_436
>>>> Convert ReLU: activation_437 --> activation_437
>>>> Convert ReLU: activation_438 --> activation_438
>>>> Convert ReLU: activation_439 --> activation_439
>>>> Convert ReLU: activation_440 --> activation_440
>>>> Convert ReLU: activation_441 --> activation_441
>>>> Convert ReLU: activation_443 --> activation_443
>>>> Convert ReLU: activation_444 --> activation_444
>>>> Convert ReLU: activation_445 --> activation_445
>>>> Convert ReLU: activation_447 --> activation_447
>>>> Convert ReLU: activation_448 --> activation_448
>>>> Convert ReLU: activation_449 --> activation_449
>>>> Convert ReLU: activation_450 --> activation_450
>>>> Convert ReLU: activation_45

In [24]:
IMAGE_SIZE = 112

img = torch.randn(3, 3, IMAGE_SIZE, IMAGE_SIZE)
img =img.numpy()

# transform= transforms.Compose([
#             transforms.Resize((112 , 112)),  # 이미지 크기 조정
#             transforms.ToTensor(),
#         ])
# img = transform(img)


basic_model(img)

ValueError: Input 0 of layer "GhostNet" is incompatible with the layer: expected shape=(None, 112, 112, 3), found shape=(3, 3, 112, 112)

In [10]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform= transforms.Compose([
            transforms.Resize((112 , 112)),  # 이미지 크기 조정
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])


# image_path =  "/home/dang/experiment/datasets/meta_datasets/mini_imagenet/test/n02110063/05408.png"
# image = Image.open(image_path).convert('RGB')
# # image_input = preprocess(image).unsqueeze(0).to(device)
# image_input = transform(image)
IMAGE_SIZE = 112
model = basic_model(image_size=IMAGE_SIZE, width=1, dropout=0., args=None)
img = torch.randn(3, 3, IMAGE_SIZE, IMAGE_SIZE)
display(model(img))

ValueError: The first argument to `Layer.call` must always be passed.

In [ ]:
from train import Train
import losses, train, GhostFaceNets, GhostFaceNets_with_Bias
import torchvision.transforms as transforms
import sys

import os
from tensorflow import keras
import losses, train, GhostFaceNets, GhostFaceNets_with_Bias
import tensorflow as tf
import keras_cv_attention_models


sys.path.append('/home/dang/experiment/Face_Attribute')
from Face_dataset import Dataload


img_path = '/home/dang/experiment/Face_Attribute/dataset'
img_csv = '/home/dang/experiment/Face_Attribute/dataset/final_data_exep3.csv'


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform= transforms.Compose([
            transforms.Resize((112 , 112)),  # 이미지 크기 조정
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

train_loader = Dataload(csv_file=img_csv, img_path = img_path, transform = transform, batch_size = 14, state='train')
val_loader = Dataload(csv_file=img_csv, img_path = img_path, transform = transform, batch_size = 14 , state='valid')
test_loader = Dataload(csv_file=img_csv, img_path = img_path, transform = transform, batch_size = 14, state='test')



# Strides of 2
basic_model = buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model)
basic_model.summary()
# basic_model.trainable = True

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: stack4_se_relu --> stack4_se_prelu
>>>> Convert ReLU: stack5_se_relu --> stack5_se_prelu
>>>> Convert ReLU: stack10_se_relu --> stack10_se_prelu
>>>> Convert ReLU: stack11_se_relu --> stack11_se_prelu
>>>> Convert ReLU: stack12_se_relu --> stack12_se_prelu
>>>> Convert ReLU: stack14_se_relu --> stack14_se_prelu
>>>> Convert ReLU: stack16_se_relu --> stack16_se_prelu
Model: "ghostnetv2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_pad (ZeroPadding2D)     

In [ ]:
# Pretrain model
from PIL import Image
device = "cpu"
 
image_path =  "/home/dang/experiment/datasets/meta_datasets/mini_imagenet/test/n02110063/05408.png"
image = Image.open(image_path).convert('RGB')
# image_input = preprocess(image).unsqueeze(0).to(device)
image_input = transform(image)
image_input = image_input.permute(1, 2, 0)
# print(image_input.size())
#### model reload
# if model is None and basic_model is None:
# model = os.path.join("checkpoints", save_path)
# embedding_layer = basic_model if basic_model is not None else __search_embedding_layer__(model)
# self.basic_model = keras.models.Model(self.model.inputs[0], self.model.layers[embedding_layer].output)
# self.inited_from_model = True

# print(">>>> Try reload from:", model)


# a = model(image_input)
# a

AttributeError: 'torch.Size' object has no attribute 'as_list'